# Accessing LLMs through APIs

In this notebook, we'll learn how to access powerful Large Language Models (LLMs) through their respective APIs. This is often the most practical approach for using state-of-the-art models without the computational overhead of running them locally.

## Learning Objectives
- Set up API keys securely using environment variables
- Understand best practices for API key management (using .env and .gitignore)
- Connect to OpenAI's models (like GPT-4)
- Connect to DeepSeek's models
- Perform basic text generation, chat, and completion tasks
- Compare results across different providers and models

## 1. API Key Management: Best Practices

When working with LLM APIs, you'll need to manage API keys securely. Here's how to do it properly:

### Why API Key Security Matters
- API keys provide access to paid services (costs can accumulate)
- Keys can be misused if stolen or leaked
- Most services have rate limits and quotas tied to keys
- Companies may require internal security compliance

### Secure API Key Management
1. **NEVER hardcode API keys in your code**
2. **NEVER commit API keys to version control**
3. **ALWAYS use environment variables or config files outside version control**
4. **Set appropriate usage limits on your API keys**

## 2. Setting Up Environment Variables

We'll use the `.env` file pattern with the `python-dotenv` library:

1. Create a `.env` file to store your API keys
2. Add the `.env` file to your `.gitignore`
3. Use `python-dotenv` to load the environment variables

Let's set up a `.env` file:

In [ ]:
# First, check if python-dotenv is installed
import sys
import subprocess

try:
    import dotenv
    print(f"✅ python-dotenv is installed (version: {dotenv.__version__})")
except ImportError:
    print("⚠️ python-dotenv not found. Installing...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "python-dotenv"])
    import dotenv
    print(f"✅ python-dotenv installed (version: {dotenv.__version__})")

# Create a .env.example file (this is safe to commit)
env_example = """# API Keys for LLM Services
# Copy this file to .env and fill in your API keys
# NEVER commit your .env file to version control!

# OpenAI API
OPENAI_API_KEY=your_openai_api_key_here

# DeepSeek API
DEEPSEEK_API_KEY=your_deepseek_api_key_here
DEEPSEEK_API_BASE=https://api.deepseek.com/v1

# Optional: Other API providers
ANTHROPIC_API_KEY=your_anthropic_api_key_here
COHERE_API_KEY=your_cohere_api_key_here

# Model Configuration
DEFAULT_MODEL=gpt-3.5-turbo
MAX_TOKENS=1000
TEMPERATURE=0.7
"""

with open('.env.example', 'w') as f:
    f.write(env_example)
    
print("📄 Created .env.example file")
print("ℹ️  This file is a template and safe to commit to version control")

# Check if .env exists, create if not
import os
if not os.path.exists('.env'):
    print("\n⚠️ No .env file found. Creating from example...")
    with open('.env', 'w') as f:
        f.write(env_example)
    print("📄 Created .env file")
    print("⚠️ Please edit the .env file with your actual API keys before proceeding")
else:
    print("\n✅ .env file already exists")

## 3. Setting Up .gitignore

To ensure your API keys aren't committed to version control, you need to add `.env` to your `.gitignore` file:

In [ ]:
# Create or update .gitignore to include .env
gitignore_content = """
# API Keys and Environment Variables
.env
*.env
.env.local

# Python
__pycache__/
*.py[cod]
*$py.class
*.so
.Python
env/
build/
develop-eggs/
dist/
downloads/
eggs/
.eggs/
lib/
lib64/
parts/
sdist/
var/
*.egg-info/
.installed.cfg
*.egg

# Jupyter Notebook
.ipynb_checkpoints

# Virtual Environment
venv/
ENV/

# IDE
.idea/
.vscode/
*.swp
*.swo

# OS-specific
.DS_Store
Thumbs.db
"""

# Check if .gitignore exists and update it
gitignore_path = '.gitignore'
if os.path.exists(gitignore_path):
    # Read existing content
    with open(gitignore_path, 'r') as f:
        existing_content = f.read()
    
    # Check if .env is already in .gitignore
    if '.env' in existing_content:
        print("✅ .env is already in .gitignore")
    else:
        # Append to existing .gitignore
        with open(gitignore_path, 'a') as f:
            f.write("\n# API Keys and Environment Variables\n.env\n*.env\n.env.local\n")
        print("✅ Added .env to existing .gitignore")
else:
    # Create new .gitignore
    with open(gitignore_path, 'w') as f:
        f.write(gitignore_content)
    print("📄 Created .gitignore file with .env excluded")

print("\n⚠️ IMPORTANT: .env file containing real API keys will NOT be committed to Git")
print("👉 Only the .env.example template (without real keys) should be committed")

## 4. Loading API Keys from Environment Variables

Now, let's load our API keys from the .env file:

In [ ]:
# Load environment variables from .env file
from dotenv import load_dotenv
import os

# Load variables from .env file into environment
load_dotenv()

# Get API keys from environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
deepseek_api_base = os.getenv("DEEPSEEK_API_BASE", "https://api.deepseek.com/v1")

# Function to check key validity (basic check)
def check_api_key(key, provider):
    if not key or key == f"your_{provider}_api_key_here":
        return False
    return True

# Check if API keys are available
print("🔑 API Key Status:\n")

if check_api_key(openai_api_key, "openai"):
    print("✅ OpenAI API key found")
else:
    print("❌ OpenAI API key not found or not set")
    print("   Please edit your .env file and add your OpenAI API key")
    print("   Get a key at: https://platform.openai.com/api-keys")

if check_api_key(deepseek_api_key, "deepseek"):
    print("✅ DeepSeek API key found")
else:
    print("❌ DeepSeek API key not found or not set")
    print("   Please edit your .env file and add your DeepSeek API key")
    print("   Get a key at: https://platform.deepseek.com/")

print("\nℹ️ You can proceed with the sections for which you have valid API keys")

## 5. Connecting to OpenAI API

Let's start with OpenAI, which provides models like GPT-3.5 and GPT-4:

In [ ]:
# Install OpenAI library if not already installed
try:
    import openai
    print(f"✅ OpenAI library is installed (version: {openai.__version__})")
except ImportError:
    print("⚠️ OpenAI library not found. Installing...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "openai"])
    import openai
    print(f"✅ OpenAI library installed (version: {openai.__version__})")

# Set up OpenAI client
from openai import OpenAI

# Skip if API key not found
if not check_api_key(openai_api_key, "openai"):
    print("⚠️ Skipping OpenAI examples due to missing API key")
else:
    # Initialize the client
    client = OpenAI(api_key=openai_api_key)
    
    print("🚀 OpenAI API Connection Established")
    
    # Get available models
    try:
        models = client.models.list()
        print("\n📋 Available Models:")
        # Display only GPT models
        gpt_models = [model.id for model in models.data if "gpt" in model.id.lower()]
        for i, model in enumerate(sorted(gpt_models), 1):
            print(f"   {i}. {model}")
        
        # Display counts by type
        model_prefixes = {}
        for model in models.data:
            prefix = model.id.split("-")[0]
            model_prefixes[prefix] = model_prefixes.get(prefix, 0) + 1
        
        print("\n📊 Model Families:")
        for prefix, count in model_prefixes.items():
            print(f"   {prefix}: {count} models")
            
    except Exception as e:
        print(f"❌ Error listing models: {e}")
        print("   This could be due to an invalid API key or connection issue")

## 6. Basic Text Generation with OpenAI

Let's perform some basic text generation tasks with OpenAI models:

In [ ]:
# Simple text generation with OpenAI
if not check_api_key(openai_api_key, "openai"):
    print("⚠️ Skipping OpenAI examples due to missing API key")
else:
    # Define a simple financial prompt
    financial_prompt = "Explain what an ETF is in simple terms and list three popular ones."
    
    print("🤖 Testing OpenAI Text Generation\n")
    print(f"Prompt: {financial_prompt}\n")
    
    # Default to gpt-3.5-turbo if no other model specified
    model = os.getenv("DEFAULT_MODEL", "gpt-3.5-turbo")
    
    try:
        # Create a completion
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a financial expert providing clear and concise information."},
                {"role": "user", "content": financial_prompt}
            ],
            max_tokens=300,
            temperature=0.7
        )
        
        # Display the response
        print(f"✅ Response from {model}:\n")
        print(response.choices[0].message.content)
        
        # Display token usage
        print("\n📊 Token Usage:")
        print(f"   Prompt tokens: {response.usage.prompt_tokens}")
        print(f"   Completion tokens: {response.usage.completion_tokens}")
        print(f"   Total tokens: {response.usage.total_tokens}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

## 7. Financial Analysis Example with OpenAI

Let's try a more complex financial analysis task:

In [ ]:
# Financial analysis example
if not check_api_key(openai_api_key, "openai"):
    print("⚠️ Skipping OpenAI examples due to missing API key")
else:
    # Financial analysis prompt
    financial_analysis_prompt = """
    Analyze the following quarterly financial data for a technology company:
    
    Revenue: $10.2B (up 12% YoY)
    Operating Income: $3.8B (up 8% YoY)
    Net Income: $2.9B (up 5% YoY)
    EPS: $1.45 (up 7% YoY)
    Cash Flow from Operations: $4.5B (up 15% YoY)
    
    Provide a brief analysis of the company's financial health and potential concerns. 
    Then recommend whether this would be a good investment based solely on this data.
    """
    
    print("📈 Financial Analysis Example\n")
    print("Prompt: Analyzing quarterly financial data for a tech company...\n")
    
    try:
        # Create a completion with more detailed system prompt
        response = client.chat.completions.create(
            model="gpt-4" if "gpt-4" in str(gpt_models) else "gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": """You are a financial analyst with expertise in technology companies.
                 Provide insightful analysis that considers growth rates, profitability, and cash flow.
                 Be balanced, highlighting both strengths and potential concerns."""},
                {"role": "user", "content": financial_analysis_prompt}
            ],
            max_tokens=500,
            temperature=0.5  # Lower temperature for more focused analysis
        )
        
        # Display the response
        model_used = "gpt-4" if "gpt-4" in str(gpt_models) else "gpt-3.5-turbo"
        print(f"✅ Financial Analysis from {model_used}:\n")
        print(response.choices[0].message.content)
        
        # Display token usage
        print("\n📊 Token Usage:")
        print(f"   Prompt tokens: {response.usage.prompt_tokens}")
        print(f"   Completion tokens: {response.usage.completion_tokens}")
        print(f"   Total tokens: {response.usage.total_tokens}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

## 8. Connecting to DeepSeek API

Now let's explore DeepSeek's models. DeepSeek is a newer provider that offers competitive LLMs:

In [ ]:
# Install requests library if not already installed
try:
    import requests
    print(f"✅ Requests library is installed")
except ImportError:
    print("⚠️ Requests library not found. Installing...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "requests"])
    import requests
    print(f"✅ Requests library installed")

# Set up DeepSeek API
if not check_api_key(deepseek_api_key, "deepseek"):
    print("⚠️ Skipping DeepSeek examples due to missing API key")
else:
    import json
    import requests
    
    # Initialize headers and API base URL
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {deepseek_api_key}"
    }
    
    # Testing connection to DeepSeek API
    try:
        # Get available models (if API supports this endpoint)
        # Note: Adjust this endpoint based on DeepSeek's actual API structure
        models_url = f"{deepseek_api_base}/models"
        models_response = requests.get(models_url, headers=headers)
        
        if models_response.status_code == 200:
            models_data = models_response.json()
            print("🚀 DeepSeek API Connection Established")
            print("\n📋 Available Models:")
            for i, model in enumerate(models_data.get("data", []), 1):
                print(f"   {i}. {model.get('id')}")
        else:
            # If the models endpoint doesn't work, just confirm connection
            print("🚀 DeepSeek API Connection Established")
            print("ℹ️ Models listing not available or requires different endpoint")
            print("   Will use default model for examples")
    
    except Exception as e:
        print(f"❌ Error connecting to DeepSeek API: {e}")
        print("   This could be due to an invalid API key or connection issue")

## 9. Text Generation with DeepSeek

Let's try text generation with DeepSeek models:

In [ ]:
# Text generation with DeepSeek
if not check_api_key(deepseek_api_key, "deepseek"):
    print("⚠️ Skipping DeepSeek examples due to missing API key")
else:
    # Define the same financial prompt for comparison
    financial_prompt = "Explain what an ETF is in simple terms and list three popular ones."
    
    print("🤖 Testing DeepSeek Text Generation\n")
    print(f"Prompt: {financial_prompt}\n")
    
    try:
        # Prepare the request
        url = f"{deepseek_api_base}/chat/completions"
        
        payload = {
            "model": "deepseek-chat",  # Adjust based on available models
            "messages": [
                {"role": "system", "content": "You are a financial expert providing clear and concise information."},
                {"role": "user", "content": financial_prompt}
            ],
            "max_tokens": 300,
            "temperature": 0.7
        }
        
        # Send the request
        response = requests.post(url, headers=headers, json=payload)
        
        if response.status_code == 200:
            result = response.json()
            
            # Display the response
            print("✅ Response from DeepSeek:\n")
            print(result["choices"][0]["message"]["content"])
            
            # Display token usage if available
            if "usage" in result:
                print("\n📊 Token Usage:")
                print(f"   Prompt tokens: {result['usage'].get('prompt_tokens', 'N/A')}")
                print(f"   Completion tokens: {result['usage'].get('completion_tokens', 'N/A')}")
                print(f"   Total tokens: {result['usage'].get('total_tokens', 'N/A')}")
        else:
            print(f"❌ Error: {response.status_code}")
            print(response.text)
            
    except Exception as e:
        print(f"❌ Error: {e}")

## 10. Financial Scenario Analysis with DeepSeek

Let's try another financial analysis task with DeepSeek:

In [ ]:
# Financial scenario analysis with DeepSeek
if not check_api_key(deepseek_api_key, "deepseek"):
    print("⚠️ Skipping DeepSeek examples due to missing API key")
else:
    # Financial scenario prompt
    scenario_prompt = """
    As a financial advisor, analyze the impact of the following macroeconomic scenario:
    
    - Federal Reserve raises interest rates by 75 basis points
    - Inflation has decreased from 6.5% to 4.8% YoY
    - Unemployment remains steady at 3.7%
    - Housing market shows signs of cooling with prices down 3% MoM
    
    What investment strategy would you recommend for:
    1. A conservative retiree with a $1M portfolio
    2. An aggressive young investor with a 30-year time horizon
    """
    
    print("📈 Financial Scenario Analysis Example\n")
    print("Prompt: Analyzing impact of Fed rate hikes and inflation...\n")
    
    try:
        # Prepare the request
        url = f"{deepseek_api_base}/chat/completions"
        
        payload = {
            "model": "deepseek-chat",  # Adjust based on available models
            "messages": [
                {"role": "system", "content": "You are an experienced financial advisor who provides balanced, thoughtful investment recommendations based on macroeconomic conditions."},
                {"role": "user", "content": scenario_prompt}
            ],
            "max_tokens": 800,
            "temperature": 0.5  # Lower for more focused analysis
        }
        
        # Send the request
        response = requests.post(url, headers=headers, json=payload)
        
        if response.status_code == 200:
            result = response.json()
            
            # Display the response
            print("✅ Financial Analysis from DeepSeek:\n")
            print(result["choices"][0]["message"]["content"])
            
            # Display token usage if available
            if "usage" in result:
                print("\n📊 Token Usage:")
                print(f"   Prompt tokens: {result['usage'].get('prompt_tokens', 'N/A')}")
                print(f"   Completion tokens: {result['usage'].get('completion_tokens', 'N/A')}")
                print(f"   Total tokens: {result['usage'].get('total_tokens', 'N/A')}")
        else:
            print(f"❌ Error: {response.status_code}")
            print(response.text)
            
    except Exception as e:
        print(f"❌ Error: {e}")

## 11. Comparing OpenAI and DeepSeek

Let's directly compare responses from both providers on the same prompt:

In [ ]:
# Comparing OpenAI and DeepSeek
comparison_prompt = """
Perform a SWOT analysis (Strengths, Weaknesses, Opportunities, Threats) for a FinTech startup 
that is developing a new AI-powered robo-advisor for sustainable investing.
Provide 3-4 bullet points for each SWOT category.
"""

print("⚖️ Comparing OpenAI vs DeepSeek\n")
print(f"Prompt: {comparison_prompt.strip()}\n")

# OpenAI response
if check_api_key(openai_api_key, "openai"):
    try:
        # Create a completion
        openai_response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a strategic business consultant with expertise in FinTech."},
                {"role": "user", "content": comparison_prompt}
            ],
            max_tokens=800,
            temperature=0.7
        )
        
        # Store the response
        openai_result = openai_response.choices[0].message.content
        openai_tokens = openai_response.usage.total_tokens
        
        print("✅ OpenAI Response:")
        print(openai_result)
        print(f"\nTokens used: {openai_tokens}")
        
    except Exception as e:
        print(f"❌ OpenAI Error: {e}")
        openai_result = None
else:
    print("⚠️ Skipping OpenAI due to missing API key")
    openai_result = None

print("\n" + "-"*80 + "\n")

# DeepSeek response
if check_api_key(deepseek_api_key, "deepseek"):
    try:
        # Prepare the request
        url = f"{deepseek_api_base}/chat/completions"
        
        payload = {
            "model": "deepseek-chat",
            "messages": [
                {"role": "system", "content": "You are a strategic business consultant with expertise in FinTech."},
                {"role": "user", "content": comparison_prompt}
            ],
            "max_tokens": 800,
            "temperature": 0.7
        }
        
        # Send the request
        response = requests.post(url, headers=headers, json=payload)
        
        if response.status_code == 200:
            result = response.json()
            
            # Store the response
            deepseek_result = result["choices"][0]["message"]["content"]
            deepseek_tokens = result.get("usage", {}).get("total_tokens", "N/A")
            
            print("✅ DeepSeek Response:")
            print(deepseek_result)
            print(f"\nTokens used: {deepseek_tokens}")
            
        else:
            print(f"❌ DeepSeek Error: {response.status_code}")
            print(response.text)
            deepseek_result = None
            
    except Exception as e:
        print(f"❌ DeepSeek Error: {e}")
        deepseek_result = None
else:
    print("⚠️ Skipping DeepSeek due to missing API key")
    deepseek_result = None

# Simple comparison if both results are available
if openai_result and deepseek_result:
    print("\n" + "="*80)
    print("📊 Comparison Summary:")
    
    # Length comparison
    openai_length = len(openai_result.split())
    deepseek_length = len(deepseek_result.split())
    
    print(f"\nResponse Length (words):")
    print(f"   OpenAI: {openai_length}")
    print(f"   DeepSeek: {deepseek_length}")
    
    # Basic overlap analysis (very simple)
    openai_words = set(w.lower() for w in openai_result.split() if len(w) > 5)
    deepseek_words = set(w.lower() for w in deepseek_result.split() if len(w) > 5)
    common_words = openai_words.intersection(deepseek_words)
    
    overlap_percent = len(common_words) / len(openai_words.union(deepseek_words)) * 100
    
    print(f"\nContent Similarity:")
    print(f"   Vocabulary overlap: {overlap_percent:.1f}%")
    print(f"   Common significant terms: {', '.join(list(common_words)[:10])}" + ("..." if len(common_words) > 10 else ""))

## 12. Best Practices for API Usage

Let's summarize some best practices for working with LLM APIs:

### API Key Management

1. **Store API Keys in .env Files**
   - Create a `.env` file for your API keys
   - Add `.env` to your `.gitignore`
   - Use `python-dotenv` to load the variables

2. **Version Control Safety**
   - Provide a `.env.example` template (without real keys)
   - Commit `.env.example` to your repository
   - NEVER commit the real `.env` file

3. **Key Rotation and Security**
   - Regularly rotate your API keys (especially in production)
   - Set usage limits on your API keys
   - Use different keys for development and production

### Cost Management

1. **Monitor API Usage**
   - Track token usage for each request
   - Set budget alerts and limits
   - Use smaller models when possible

2. **Optimize Prompts**
   - Keep prompts concise but clear
   - Use system messages to set context
   - Adjust `max_tokens` based on your needs

3. **Caching**
   - Cache responses for identical requests
   - Implement TTL (Time-To-Live) for cached responses
   - Consider semantic caching for similar requests

### Error Handling

1. **Robust Error Handling**
   - Handle rate limits with exponential backoff
   - Implement fallbacks between models/providers
   - Log errors for debugging

2. **Validation**
   - Validate inputs before sending to API
   - Validate outputs for expected format
   - Implement content filtering where needed

### Advanced Usage

1. **Function Calling**
   - Use function calling for structured outputs
   - Define clear JSON schemas
   - Validate returned JSON

2. **Streaming**
   - Use streaming for better UX on longer responses
   - Process chunks as they arrive
   - Implement early stopping if needed

3. **RAG (Retrieval-Augmented Generation)**
   - Combine APIs with your own data sources
   - Use embeddings for semantic search
   - Implement proper citation and attribution

## 13. Project: Financial News Sentiment Analyzer

Let's build a simple but practical example: a financial news sentiment analyzer using the OpenAI or DeepSeek API:

In [ ]:
# Financial News Sentiment Analyzer

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time
from datetime import datetime

# Check if we have at least one API key available
have_api = check_api_key(openai_api_key, "openai") or check_api_key(deepseek_api_key, "deepseek")

if not have_api:
    print("⚠️ No valid API keys found. Please add at least one API key to your .env file.")
else:
    print("🔍 Financial News Sentiment Analyzer\n")
    
    # Sample financial news headlines
    financial_news = [
        {"date": "2025-06-01", "headline": "Tesla reports record quarterly deliveries, shares jump 8%"},
        {"date": "2025-06-02", "headline": "Fed signals potential interest rate cut in September"},
        {"date": "2025-06-03", "headline": "Amazon acquires AI startup for $2.5 billion"},
        {"date": "2025-06-04", "headline": "Inflation rises to 5.2%, exceeding economist expectations"},
        {"date": "2025-06-05", "headline": "Tech stocks tumble amid renewed regulatory concerns"},
        {"date": "2025-06-06", "headline": "Bitcoin surpasses $100,000 for the first time"},
        {"date": "2025-06-07", "headline": "Major bank announces 5,000 layoffs amid restructuring"},
        {"date": "2025-06-08", "headline": "Apple unveils new product line at annual conference"},
        {"date": "2025-06-09", "headline": "Oil prices drop 10% as OPEC increases production"},
        {"date": "2025-06-10", "headline": "Consumer confidence index hits 5-year high"}
    ]
    
    # Create DataFrame
    news_df = pd.DataFrame(financial_news)
    print(f"📰 Analyzing {len(news_df)} financial news headlines...\n")
    
    # Function to analyze sentiment with OpenAI
    def analyze_with_openai(headline):
        if not check_api_key(openai_api_key, "openai"):
            return {"sentiment": "N/A", "score": 0, "rationale": "API key not available"}
        
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a financial analyst. Analyze the sentiment of financial news headlines."},
                    {"role": "user", "content": f"""Analyze the sentiment of this financial news headline:
                    
                    "{headline}"
                    
                    Provide your response in JSON format with the following fields:
                    - sentiment: Either "positive", "negative", or "neutral"
                    - score: A number from -1.0 (very negative) to 1.0 (very positive)
                    - rationale: A brief explanation of your reasoning (max 20 words)"""}
                ],
                max_tokens=150,
                temperature=0.3
            )
            
            # Extract the JSON response
            content = response.choices[0].message.content
            # Use regex to extract JSON
            json_match = re.search(r'\{[^}]+\}', content.replace('\n', ' '))
            if json_match:
                import json
                result = json.loads(json_match.group(0))
            else:
                # Fallback parsing for non-JSON responses
                if "positive" in content.lower():
                    sentiment = "positive"
                    score = 0.7
                elif "negative" in content.lower():
                    sentiment = "negative"
                    score = -0.7
                else:
                    sentiment = "neutral"
                    score = 0
                result = {
                    "sentiment": sentiment,
                    "score": score,
                    "rationale": content[:50] + "..."
                }
            
            return result
            
        except Exception as e:
            print(f"Error analyzing '{headline}': {e}")
            return {"sentiment": "error", "score": 0, "rationale": str(e)[:50]}
    
    # Function to analyze sentiment with DeepSeek
    def analyze_with_deepseek(headline):
        if not check_api_key(deepseek_api_key, "deepseek"):
            return {"sentiment": "N/A", "score": 0, "rationale": "API key not available"}
        
        try:
            url = f"{deepseek_api_base}/chat/completions"
            
            payload = {
                "model": "deepseek-chat",
                "messages": [
                    {"role": "system", "content": "You are a financial analyst. Analyze the sentiment of financial news headlines."},
                    {"role": "user", "content": f"""Analyze the sentiment of this financial news headline:
                    
                    "{headline}"
                    
                    Provide your response in JSON format with the following fields:
                    - sentiment: Either "positive", "negative", or "neutral"
                    - score: A number from -1.0 (very negative) to 1.0 (very positive)
                    - rationale: A brief explanation of your reasoning (max 20 words)"""}
                ],
                "max_tokens": 150,
                "temperature": 0.3
            }
            
            response = requests.post(url, headers=headers, json=payload)
            
            if response.status_code == 200:
                result = response.json()
                content = result["choices"][0]["message"]["content"]
                
                # Use regex to extract JSON
                json_match = re.search(r'\{[^}]+\}', content.replace('\n', ' '))
                if json_match:
                    import json
                    result = json.loads(json_match.group(0))
                else:
                    # Fallback parsing for non-JSON responses
                    if "positive" in content.lower():
                        sentiment = "positive"
                        score = 0.7
                    elif "negative" in content.lower():
                        sentiment = "negative"
                        score = -0.7
                    else:
                        sentiment = "neutral"
                        score = 0
                    result = {
                        "sentiment": sentiment,
                        "score": score,
                        "rationale": content[:50] + "..."
                    }
                
                return result
            else:
                return {"sentiment": "error", "score": 0, "rationale": f"API error: {response.status_code}"}
                
        except Exception as e:
            print(f"Error analyzing '{headline}': {e}")
            return {"sentiment": "error", "score": 0, "rationale": str(e)[:50]}
    
    # Choose which API to use based on available keys
    if check_api_key(openai_api_key, "openai"):
        analyze_sentiment = analyze_with_openai
        api_used = "OpenAI"
    elif check_api_key(deepseek_api_key, "deepseek"):
        analyze_sentiment = analyze_with_deepseek
        api_used = "DeepSeek"
    else:
        print("⚠️ No valid API keys found. Cannot proceed with analysis.")
        analyze_sentiment = None
        api_used = None
    
    # Process headlines if we have a valid API
    if analyze_sentiment:
        results = []
        
        print(f"🔄 Processing with {api_used} API...")
        for i, row in news_df.iterrows():
            print(f"   Analyzing ({i+1}/{len(news_df)}): {row['headline'][:40]}...")
            result = analyze_sentiment(row['headline'])
            
            # Add result to our data
            results.append({
                "date": row['date'],
                "headline": row['headline'],
                "sentiment": result.get("sentiment", "N/A"),
                "score": result.get("score", 0),
                "rationale": result.get("rationale", "N/A")
            })
            
            # Sleep briefly to avoid rate limits
            time.sleep(0.5)
        
        # Create results DataFrame
        results_df = pd.DataFrame(results)
        
        # Display results
        print("\n📊 Sentiment Analysis Results:\n")
        for i, row in results_df.iterrows():
            # Create an emoji indicator
            if row['sentiment'] == "positive":
                emoji = "📈"
                color = "\033[92m"  # Green
            elif row['sentiment'] == "negative":
                emoji = "📉"
                color = "\033[91m"  # Red
            else:
                emoji = "➡️"
                color = "\033[93m"  # Yellow
            
            # Reset color
            reset = "\033[0m"
            
            print(f"{row['date']} | {color}{row['sentiment'].upper()} ({row['score']:+.2f}){reset} {emoji}")
            print(f"   {row['headline']}")
            print(f"   Rationale: {row['rationale']}")
            print("-" * 80)
        
        # Create a visualization
        plt.figure(figsize=(12, 8))
        
        # Convert date to datetime for proper sorting
        results_df['date'] = pd.to_datetime(results_df['date'])
        results_df = results_df.sort_values('date')
        
        # Plot 1: Sentiment score timeline
        plt.subplot(2, 1, 1)
        sns.lineplot(x='date', y='score', data=results_df, marker='o', linewidth=2)
        plt.axhline(y=0, color='r', linestyle='-', alpha=0.3)
        plt.title(f'Financial News Sentiment Timeline ({api_used} Analysis)', fontsize=14)
        plt.ylabel('Sentiment Score (-1 to +1)')
        plt.ylim(-1.1, 1.1)
        
        # Annotate points with headlines (shortened)
        for i, row in results_df.iterrows():
            short_headline = row['headline'][:20] + "..." if len(row['headline']) > 20 else row['headline']
            plt.annotate(short_headline, (row['date'], row['score']), 
                        xytext=(5, 5), textcoords='offset points', fontsize=8)
        
        # Plot 2: Sentiment distribution
        plt.subplot(2, 1, 2)
        sentiment_counts = results_df['sentiment'].value_counts()
        colors = {'positive': 'green', 'negative': 'red', 'neutral': 'orange', 'error': 'gray'}
        sns.barplot(x=sentiment_counts.index, y=sentiment_counts.values, 
                   palette=[colors.get(x, 'blue') for x in sentiment_counts.index])
        plt.title('Sentiment Distribution', fontsize=14)
        plt.ylabel('Count')
        plt.xlabel('Sentiment')
        
        # Add value labels on bars
        for i, v in enumerate(sentiment_counts.values):
            plt.text(i, v + 0.1, str(v), ha='center')
        
        plt.tight_layout()
        plt.savefig('financial_news_sentiment.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        # Save results to CSV
        results_df.to_csv('financial_news_sentiment_results.csv', index=False)
        print("\n💾 Results saved to 'financial_news_sentiment_results.csv'")
        print("📊 Visualization saved to 'financial_news_sentiment.png'")
        
        # Calculate market sentiment indicator
        avg_sentiment = results_df['score'].mean()
        sentiment_direction = "BULLISH" if avg_sentiment > 0.3 else "BEARISH" if avg_sentiment < -0.3 else "NEUTRAL"
        
        print(f"\n📈 Market Sentiment Indicator: {sentiment_direction}")
        print(f"   Average Sentiment Score: {avg_sentiment:.2f}")
        
        # Most positive and negative headlines
        if len(results_df) > 0:
            most_positive = results_df.loc[results_df['score'].idxmax()]
            most_negative = results_df.loc[results_df['score'].idxmin()]
            
            print(f"\n📰 Most Positive News: {most_positive['headline']}")
            print(f"   Score: +{most_positive['score']:.2f}")
            
            print(f"\n📰 Most Negative News: {most_negative['headline']}")
            print(f"   Score: {most_negative['score']:.2f}")

## 14. Summary and Next Steps

### What We've Learned
- How to securely manage API keys using `.env` files
- How to properly exclude sensitive files with `.gitignore`
- How to connect to and use OpenAI's API
- How to connect to and use DeepSeek's API
- How to compare responses from different providers
- How to build a practical financial news sentiment analyzer

### Best Practices Recap
- Never hardcode API keys in your code
- Always use environment variables
- Add `.env` to your `.gitignore`
- Create `.env.example` templates for collaboration
- Implement robust error handling
- Monitor and manage API usage costs

### Next Steps
1. **Explore More Models**: Try different models from each provider
2. **Build Financial Applications**: Create more sophisticated financial tools
3. **Implement RAG**: Combine LLMs with your own financial data
4. **Fine-tune Models**: Train models on financial domain data
5. **Deploy Applications**: Create user interfaces for your LLM applications

### Additional Resources
- [OpenAI API Documentation](https://platform.openai.com/docs/api-reference)
- [DeepSeek API Documentation](https://platform.deepseek.com/)
- [LangChain Library](https://python.langchain.com/) - For building more complex LLM applications
- [LlamaIndex](https://docs.llamaindex.ai/) - For connecting LLMs to your custom data
- [API Security Best Practices](https://owasp.org/www-project-api-security/)